In [11]:
rule = "If a student has high grades, is a student leader, and applied, then they are Eligible."

juan_facts = ["HighGrades", "Applied"]
maria_facts = ["HighGrades", "DidNotApply"]
carlo_facts = ["StudentLeader", "NoHighGrades"]
ana_facts = ["HighGrades", "StudentLeader", "Applied"]

print("Rule:", rule)
print("Juan's Facts:", juan_facts)
print("Maria's Facts:", maria_facts)
print("Carlos's Facts:", carlo_facts)
print("Ana's Facts:", ana_facts)

Rule: If a student has high grades, is a student leader, and applied, then they are Eligible.
Juan's Facts: ['HighGrades', 'Applied']
Maria's Facts: ['HighGrades', 'DidNotApply']
Carlos's Facts: ['StudentLeader', 'NoHighGrades']
Ana's Facts: ['HighGrades', 'StudentLeader', 'Applied']


In [12]:
# Unification
substitution_set = {"x": "Ana"}

print("Rule: Eligible(x) -> Scholarship(x)")
print("Fact: Eligible(Ana)")
print("Substitution set:", substitution_set)

print("\nAfter unification, the rule becomes: Eligible(Ana) -> Scholarship(Ana)")

Rule: Eligible(x) -> Scholarship(x)
Fact: Eligible(Ana)
Substitution set: {'x': 'Ana'}

After unification, the rule becomes: Eligible(Ana) -> Scholarship(Ana)


In [27]:
#Forward Chaining
rules = [
    ("HighGrades", "StudentLeader", "Applied", "Eligible"),
    ("Eligible", "Scholarship"),
    ("Scholarship", "FinancialSupport")
]

juan_facts = set(["HighGrades", "Applied"])
maria_facts = set(["HighGrades", "DidNotApply"])
carlo_facts = set(["StudentLeader", "NoHighGrades"])
ana_facts = set(["HighGrades", "StudentLeader", "Applied"])

def forward_chaining(facts, rules):
    new_facts = set(facts)
    inferred_facts = set()
    changed = True
    while changed:
        changed = False
        for rule in rules:
            antecedent = set(rule[:-1])
            consequent = rule[-1]
            if antecedent.issubset(new_facts) and consequent not in new_facts:
                new_facts.add(consequent)
                inferred_facts.add(consequent)
                changed = True
    return new_facts, inferred_facts

print(f"Juan's initial facts: {juan_facts}")
juan_derived_facts, juan_inferred = forward_chaining(juan_facts, rules)
print(f"Juan's derived facts: {juan_derived_facts}")
if "Eligible" in juan_derived_facts:
    print("Outcome for Juan: Eligible")
else:
    print("Outcome for Juan: Not Eligible")
if juan_inferred:
    print(f"New facts derived for Juan: {juan_inferred}")
else:
    print("New facts derived for Juan: no new derived facts")

print("------------------------------------------------")
print(f"Ana's initial facts: {ana_facts}")
ana_derived_facts, ana_inferred = forward_chaining(ana_facts, rules)
print(f"Ana's derived facts: {ana_derived_facts}")
if "Eligible" in ana_derived_facts:
    print("Outcome for Ana: Eligible")
else:
    print("Outcome for Ana: Not Eligible")
if ana_inferred:
    print(f"New facts derived for Ana: {ana_inferred}")
else:
    print("New facts derived for Ana: no new derived facts")

Juan's initial facts: {'Applied', 'HighGrades'}
Juan's derived facts: {'Applied', 'HighGrades'}
Outcome for Juan: Not Eligible
New facts derived for Juan: no new derived facts
------------------------------------------------
Ana's initial facts: {'Applied', 'StudentLeader', 'HighGrades'}
Ana's derived facts: {'Scholarship', 'HighGrades', 'FinancialSupport', 'StudentLeader', 'Applied', 'Eligible'}
Outcome for Ana: Eligible
New facts derived for Ana: {'Scholarship', 'Eligible', 'FinancialSupport'}


### Explain why Maria and Carlos are not eligible.


**Maria:**
Maria is not eligible because her facts do not meet all the conditions for eligibility. She has 'High Grades' but has 'Did not Apply' thus failing the condition.



**Carlos:**
Carlos is not eligible because his facts do not meet all the conditions for eligibility. He has 'Student Leader' but has 'No High Grades' thus failing the condition.


In [28]:
# Backward Chaining
rules = {
    "Eligible": [["HighGrades", "StudentLeader", "Applied"]],
    "Scholarship": [["Eligible"]],
    "FinancialSupport": [["Scholarship"]]
}

maria_facts = set(["HighGrades", "DidNotApply"])
ana_facts = set(["HighGrades", "StudentLeader", "Applied"])

maria_goal = "Eligible"
ana_goal = "FinancialSupport"

def backward_chaining(goal, facts, rules, depth=0, path=[]):
    """Applies backward chaining to check if a goal can be proven."""
    indent = "  " * depth
    print(f"{indent}Checking goal: {goal}")

    if goal in facts:
        print(f"{indent}Goal '{goal}' is a known fact.")
        return True, path + [f"Goal '{goal}' is a known fact."]

    if goal in rules:
        print(f"{indent}Found rules for goal '{goal}'. Trying to prove antecedents.")
        for rule_antecedent in rules[goal]:
            print(f"{indent}Trying rule with antecedent: {rule_antecedent}")
            all_antecedents_proven = True
            step_reasoning = [f"To prove '{goal}', trying to prove antecedent '{rule_antecedent}'"]
            for sub_goal in rule_antecedent:
                proven, sub_path = backward_chaining(sub_goal, facts, rules, depth + 1, [])
                if not proven:
                    all_antecedents_proven = False
                    step_reasoning.append(f"Failed to prove sub-goal: {sub_goal}")
                    break
                else:
                    step_reasoning.append(f"Successfully proved sub-goal: {sub_goal}")
                    step_reasoning.extend(sub_path)

            if all_antecedents_proven:
                print(f"{indent}All antecedents proven for rule leading to '{goal}'. Goal proven.")
                return True, path + step_reasoning + [f"Successfully proved '{goal}' using rule with antecedent '{rule_antecedent}'"]
            else:
                print(f"{indent}Antecedents not all proven for rule leading to '{goal}'. Trying next rule.")

    print(f"{indent}No rules or facts found to prove goal: {goal}")
    return False, path + [f"Failed to prove '{goal}'"]

print("--- Backward Chaining for Maria ---")
maria_proven, maria_reasoning = backward_chaining(maria_goal, maria_facts, rules)
print(f"\nResult for Maria: {maria_goal} is {'proven' if maria_proven else 'not proven'}")
print("\nReasoning steps for Maria:")
for step in maria_reasoning:
    print(step)

print("\n--- Backward Chaining for Ana ---")
ana_proven, ana_reasoning = backward_chaining(ana_goal, ana_facts, rules)
print(f"\nResult for Ana: {ana_goal} is {'proven' if ana_proven else 'not proven'}")
print("\nReasoning steps for Ana:")
for step in ana_reasoning:
    print(step)

--- Backward Chaining for Maria ---
Checking goal: Eligible
Found rules for goal 'Eligible'. Trying to prove antecedents.
Trying rule with antecedent: ['HighGrades', 'StudentLeader', 'Applied']
  Checking goal: HighGrades
  Goal 'HighGrades' is a known fact.
  Checking goal: StudentLeader
  No rules or facts found to prove goal: StudentLeader
Antecedents not all proven for rule leading to 'Eligible'. Trying next rule.
No rules or facts found to prove goal: Eligible

Result for Maria: Eligible is not proven

Reasoning steps for Maria:
Failed to prove 'Eligible'

--- Backward Chaining for Ana ---
Checking goal: FinancialSupport
Found rules for goal 'FinancialSupport'. Trying to prove antecedents.
Trying rule with antecedent: ['Scholarship']
  Checking goal: Scholarship
  Found rules for goal 'Scholarship'. Trying to prove antecedents.
  Trying rule with antecedent: ['Eligible']
    Checking goal: Eligible
    Found rules for goal 'Eligible'. Trying to prove antecedents.
    Trying rule w

In [6]:
#validity check
import pandas as pd
data = {
    'HighGrades (A)': [False, False, False, False, True, True, True, True],
    'StudentLeader (B)': [False, False, True, True, False, False, True, True],
    'Applied (C)': [False, True, False, True, False, True, False, True]
}
truth_table = pd.DataFrame(data)

truth_table['Eligible'] = truth_table['HighGrades (A)'] & truth_table['StudentLeader (B)'] & truth_table['Applied (C)']

print("--- Truth Table for Eligibility Rule ---")
display(truth_table)


student_facts = {
    'Juan': {'HighGrades': True, 'StudentLeader': False, 'Applied': True},
    'Maria': {'HighGrades': True, 'StudentLeader': False, 'Applied': False},
    'Carlos': {'HighGrades': False, 'StudentLeader': True, 'Applied': False},
    'Ana': {'HighGrades': True, 'StudentLeader': True, 'Applied': True}
}

actual_outcomes = {
    'Juan': 'Not Eligible',
    'Maria': 'Not Eligible',
    'Carlos': 'Not Eligible',
    'Ana': 'Eligible'
}

print("\n--- Evaluating Each Student ---")

for student, facts in student_facts.items():
    student_conditions = {
        'HighGrades (A)': facts['HighGrades'],
        'StudentLeader (B)': facts['StudentLeader'],
        'Applied (C)': facts['Applied']
    }

    matching_row = truth_table[
        (truth_table['HighGrades (A)'] == student_conditions['HighGrades (A)']) &
        (truth_table['StudentLeader (B)'] == student_conditions['StudentLeader (B)']) &
        (truth_table['Applied (C)'] == student_conditions['Applied (C)'])
    ]

    if not matching_row.empty:
        expected_outcome = 'Eligible' if matching_row['Eligible'].iloc[0] else 'Not Eligible'
    else:
        expected_outcome = 'Could not determine from truth table'

    actual_outcome = actual_outcomes[student]

    print(f"\nStudent: {student}")
    print(f"  Truth values for conditions: HighGrades={facts['HighGrades']}, StudentLeader={facts['StudentLeader']}, Applied={facts['Applied']}")
    print(f"  Expected outcome from truth table: {expected_outcome}")
    print(f"  Actual outcome from Part 2: {actual_outcome}")
    print(f"  Match: {'Yes' if expected_outcome == actual_outcome else 'No'}")

    if expected_outcome == actual_outcome:
        print(f"  Explanation: The truth table and forward chaining results match. {student}'s facts correspond to the row in the truth table where Eligible is {expected_outcome}.")
    else:
         print(f"  Explanation: The truth table and forward chaining results do not match. This indicates a potential issue in the truth table construction or the forward chaining logic.")


--- Truth Table for Eligibility Rule ---


,HighGrades (A),StudentLeader (B),Applied (C),Eligible
0,False,False,False,False
1,False,False,True,False
2,False,True,False,False
3,False,True,True,False
4,True,False,False,False
5,True,False,True,False
6,True,True,False,False
7,True,True,True,True



--- Evaluating Each Student ---

Student: Juan
  Truth values for conditions: HighGrades=True, StudentLeader=False, Applied=True
  Expected outcome from truth table: Not Eligible
  Actual outcome from Part 2: Not Eligible
  Match: Yes
  Explanation: The truth table and forward chaining results match. Juan's facts correspond to the row in the truth table where Eligible is Not Eligible.

Student: Maria
  Truth values for conditions: HighGrades=True, StudentLeader=False, Applied=False
  Expected outcome from truth table: Not Eligible
  Actual outcome from Part 2: Not Eligible
  Match: Yes
  Explanation: The truth table and forward chaining results match. Maria's facts correspond to the row in the truth table where Eligible is Not Eligible.

Student: Carlos
  Truth values for conditions: HighGrades=False, StudentLeader=True, Applied=False
  Expected outcome from truth table: Not Eligible
  Actual outcome from Part 2: Not Eligible
  Match: Yes
  Explanation: The truth table and forward cha

***Reflection:***

Logical reasoning is important in AI and rel-life decision making because it provides a systematic and structured way to process information and create a conclusion to solve the problem. In AI, it enables systems to understand complex relationships, infer new knowledge form existing facts, and make decisions based on predefined rules and data. Logical reasoning helps individuals analyze situations, evaluate options, identify possible consequences, and make proper judgements. Without logical reasoning, AI systems would struggle to exhibit intelligent behavior, and human decision-making would be prone to errors and inconsistencies.